## imports

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
import random 

## reading inputs 


In [18]:
df=pd.read_csv("mnist_train.csv")
test_df=pd.read_csv("mnist_test.csv")

In [19]:
train_data = df.values  
test_data=test_df.values
train_tensor = torch.tensor(train_data, dtype=torch.float32)
test_tensor = torch.tensor(test_data, dtype=torch.float32)
print(train_tensor)

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [6., 0., 0.,  ..., 0., 0., 0.],
        [8., 0., 0.,  ..., 0., 0., 0.]])


In [20]:
#shuffling the training dataset
random.seed(9)
shuffled_indices = torch.randperm(len(train_tensor))
train_tensor = train_tensor[shuffled_indices]

print(train_tensor)

tensor([[9., 0., 0.,  ..., 0., 0., 0.],
        [8., 0., 0.,  ..., 0., 0., 0.],
        [7., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [2., 0., 0.,  ..., 0., 0., 0.],
        [6., 0., 0.,  ..., 0., 0., 0.],
        [8., 0., 0.,  ..., 0., 0., 0.]])


In [21]:
#dividing features and labels from the dataset.
features=train_tensor[:,1:]
labels=train_tensor[:,0].long()

test_features=test_tensor[:,1:]
test_labels=test_tensor[:,0].long()

print(features)
print(labels)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([9, 8, 7,  ..., 2, 6, 8])


## intializing weights/ layers 


In [22]:
g=torch.Generator().manual_seed(9)

w1=torch.randn(784,256,generator=g)* (2 / 784)**0.5#hidden layer 1
b1=torch.zeros(256)


w2=torch.randn(256,10,generator=g)* (2 / 256)**0.5#hidden layer 2
b2=torch.zeros(10)





parameters=[w1,b1,w2,b2]
for p in parameters:
    p.requires_grad=True

In [35]:
epoch=100#800 total

for i in range(epoch):
    layer1 = torch.relu(features @ w1 + b1)
    logits = layer1 @ w2 + b2
    loss = F.cross_entropy(logits, labels)
    
    
    for p in parameters:
        p.grad=None
    loss.backward()

    
    lr=0.0005#500 0.1 and 200 0.01 and 100 0.005
    for p in parameters:
        p.data-=lr*p.grad
    if i%10==0:
            print(i,'->',loss.item())
print(loss.item())


0 -> 0.36886918544769287
10 -> 0.3684919476509094
20 -> 0.36811912059783936
30 -> 0.36775174736976624
40 -> 0.36738744378089905
50 -> 0.3670269846916199
60 -> 0.36667150259017944
70 -> 0.3663199245929718
80 -> 0.36597195267677307
90 -> 0.36562666296958923
0.36531907320022583


In [36]:
'''
# Load the test CSV file
test_csv_path = "mnist_test.csv"  # Replace with your test CSV file path
test_df = pd.read_csv(test_csv_path)

# Convert to PyTorch tensor
test_data = test_df.values
test_tensor = torch.tensor(test_data, dtype=torch.float32)

# Normalize pixel values to [0, 1]
test_tensor[:, 1:] /= 255.0

# Separate features (pixel values) and labels (if available)
test_features = test_tensor[:, 1:]  # Features (pixel values)
test_labels = test_tensor[:, 0].long()  # Labels (if available, otherwise ignore)

'''

with torch.no_grad():
    layer1 = F.relu(test_features @ w1 + b1)
    test_logits = layer1 @ w2 + b2
    predictions = torch.argmax(test_logits, dim=1)
    accuracy = (predictions == test_labels).float().mean()
    print(f"Accuracy: {accuracy.item() * 100:.2f}%")

Accuracy: 89.29%


In [37]:
with torch.no_grad():
    # Forward pass
    layer1 = F.relu(test_features @ w1 + b1)
    test_logits = layer1 @ w2 + b2
    predictions = torch.argmax(test_logits, dim=1)
    
    # Total accuracy
    total_accuracy = (predictions == test_labels).float().mean().item() * 100
    
    # Per-digit accuracy
    digit_accuracies = {}
    for digit in range(10):
        mask = (test_labels == digit)
        correct = (predictions[mask] == test_labels[mask]).sum().item()
        total = mask.sum().item()
        digit_accuracies[digit] = (correct / total) * 100 if total > 0 else 0.0

# Print results
print(f"\nTotal Accuracy: {total_accuracy:.2f}%")
print("Per-Digit Accuracy:")
for digit in range(10):
    print(f"Digit {digit}: {digit_accuracies[digit]:.2f}%")


Total Accuracy: 89.29%
Per-Digit Accuracy:
Digit 0: 83.98%
Digit 1: 93.66%
Digit 2: 90.70%
Digit 3: 87.13%
Digit 4: 89.31%
Digit 5: 83.97%
Digit 6: 90.61%
Digit 7: 90.76%
Digit 8: 91.48%
Digit 9: 90.09%
